In [1]:
from gprmaxui.commands import *
from gprmaxui import GprMaxModel
import panel as pn
pn.extension('vtk')  # Necessary to enable PyVista plots in Jupyter

In [2]:
import logging
logger = logging.getLogger('pyvista')
logging.basicConfig(level=logging.ERROR)

In [3]:

# Create a GPRMax model
model = GprMaxModel(
    title="B scan from a single target buried in a dielectric half-space",
    output_folder=Path("output").absolute(),
    domain_size=DomainSize(x=0.2, y=0.2, z=0.002),
    domain_resolution=DomainResolution(dx=0.002, dy=0.002, dz=0.002),
    time_window=TimeWindow(twt=3e-9),
)
# Register model materials
model.register_materials(
    Material(id="half_space", permittivity=6, conductivity=0, permeability=1)
)

# add model geometries
box = DomainBox(
    x_min=0.0,
    y_min=0.0,
    z_min=0.0,
    x_max=0.2,
    y_max=0.145,
    z_max=0.002,
    material="half_space",
)
model.add_geometry(box)

cx = box.center().x
cy = box.center().y
cz = box.center().z
sphere = DomainSphere(cx=cx, cy=cy, cz=cz, radius=0.005, material="pec")
model.add_geometry(sphere)

# Register model sources
tx_rx_sep = 2e-2
tx = Tx(
    waveform=Waveform(wave_family="ricker", amplitude=1.0, frequency=1.5e9),
    source=HertzianDipole(polarization="z", x=0.03, y=0.15, z=0.0),
)
rx = Rx(x=tx.source.x + tx_rx_sep, y=0.15, z=0.0)

model.set_source(
    TxRxPair(
        tx=tx,
        rx=rx,
        src_steps=SrcSteps(dx=0.002, dy=0.0, dz=0.0),
        rx_steps=RxSteps(dx=0.002, dy=0.0, dz=0.0),
    )
)

In [4]:
model.plot_geometry()


=== Electromagnetic modelling software based on the Finite-Difference Time-Domain (FDTD) method 

    www.gprmax.com   __  __
     __ _ _ __  _ __|  \/  | __ ___  __
    / _` | '_ \| '__| |\/| |/ _` \ \/ /
   | (_| | |_) | |  | |  | | (_| |>  <
    \__, | .__/|_|  |_|  |_|\__,_/_/\_\
    |___/|_|
                     v3.1.7 (Big Smoke)

 Copyright (C) 2015-2023: The University of Edinburgh
 Authors: Craig Warren and Antonis Giannopoulos

 gprMax is free software: you can redistribute it and/or modify it under the
  terms of the GNU General Public License as published by the Free Software
  Foundation, either version 3 of the License, or (at your option) any later
  version.
 gprMax is distributed in the hope that it will be useful, but WITHOUT ANY
  WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR
  A PARTICULAR PURPOSE.  See the GNU General Public License for more details.
 You should have received a copy of the GNU General Public License along with
  gpr

Widget(value='<iframe src="http://localhost:36555/index.html?ui=P_0x78dbd3b5fd10_0&reconnect=auto" class="pyvi…


=== Electromagnetic modelling software based on the Finite-Difference Time-Domain (FDTD) method 

    www.gprmax.com   __  __
     __ _ _ __  _ __|  \/  | __ ___  __
    / _` | '_ \| '__| |\/| |/ _` \ \/ /
   | (_| | |_) | |  | |  | | (_| |>  <
    \__, | .__/|_|  |_|  |_|\__,_/_/\_\
    |___/|_|
                     v3.1.7 (Big Smoke)

 Copyright (C) 2015-2023: The University of Edinburgh
 Authors: Craig Warren and Antonis Giannopoulos

 gprMax is free software: you can redistribute it and/or modify it under the
  terms of the GNU General Public License as published by the Free Software
  Foundation, either version 3 of the License, or (at your option) any later
  version.
 gprMax is distributed in the hope that it will be useful, but WITHOUT ANY
  WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR
  A PARTICULAR PURPOSE.  See the GNU General Public License for more details.
 You should have received a copy of the GNU General Public License along with
  gpr

In [5]:
from IPython.display import JSON
import json
json_str = model.to_json()
json_dict = json.loads(json_str)
JSON(json_dict)

<IPython.core.display.JSON object>

In [6]:
model = GprMaxModel.from_json(json_str)
model.plot_geometry()

Widget(value='<iframe src="http://localhost:36555/index.html?ui=P_0x78dcbc080380_1&reconnect=auto" class="pyvi…

In [7]:
print(model)